In [2]:
import os
import csv

def process_all_folders(root_dir, output_csv):
    """
    Recursively scan the dataset directory structure, extract data from 
    each RAW_ACCELEROMETERS file, and combine it into one CSV with additional
    label columns extracted from the folder name.
    
    Assumed folder structure:
      root_dir/
          DriverFolder/         (e.g., D1, D2, etc.)
              TripFolder/       (folder name format: "YYYYMMDDhhmmss-Distance-Driver-Behavior-Road")
                  RAW_ACCELEROMETERS.txt (or similar)
                  (other files...)
    
    The folder name is expected to have 5 tokens separated by '-' in this order:
      Date, Distance, Driver, Behavior, Road
    """
    combined_data = []
    
    # Define CSV header: 11 sensor columns plus 5 metadata columns
    header = [
        "Timestamp_s",
        "SystemActivated",
        "AccelX_Gs",
        "AccelY_Gs",
        "AccelZ_Gs",
        "AccelX_KF_Gs",
        "AccelY_KF_Gs",
        "AccelZ_KF_Gs",
        "Roll_deg",
        "Pitch_deg",
        "Yaw_deg",
        "TripDate",
        "Distance",
        "Driver",
        "Behavior",
        "Road"
    ]
    
    # Loop through each driver folder in the root directory
    for driver_folder in os.listdir(root_dir):
        driver_folder_path = os.path.join(root_dir, driver_folder)
        if not os.path.isdir(driver_folder_path):
            continue
        
        # Loop through each trip subfolder in the driver folder
        for trip_folder in os.listdir(driver_folder_path):
            trip_folder_path = os.path.join(driver_folder_path, trip_folder)
            if not os.path.isdir(trip_folder_path):
                continue

            # Parse folder name for metadata.
            # Expected format: "YYYYMMDDhhmmss-Distance-Driver-Behavior-Road"
            tokens = trip_folder.split('-')
            if len(tokens) >= 5:
                trip_date   = tokens[0]
                distance    = tokens[1]
                driver_label = tokens[2]
                behavior    = tokens[3]
                road        = tokens[4]
            else:
                # If the folder name doesn't match the expected format, skip it.
                print(f"Skipping folder (unexpected naming): {trip_folder}")
                continue
            
            # Look for the RAW_ACCELEROMETERS file (case-insensitive search)
            for file_name in os.listdir(trip_folder_path):
                if "RAW" in file_name.upper() and "ACCELEROMETERS" in file_name.upper():
                    file_path = os.path.join(trip_folder_path, file_name)
                    print(f"Processing file: {file_path}")
                    
                    # Open the file and process each line
                    with open(file_path, 'r') as f:
                        for line in f:
                            line = line.strip()
                            if not line:
                                continue
                            parts = line.split()
                            # Expecting 11 columns per line as per dataset documentation
                            if len(parts) != 11:
                                print(f"Skipping line (unexpected number of values): {line}")
                                continue
                            try:
                                # Convert sensor data to appropriate types
                                timestamp       = float(parts[0])
                                system_activated = int(parts[1])
                                accel_x         = float(parts[2])
                                accel_y         = float(parts[3])
                                accel_z         = float(parts[4])
                                accel_x_kf      = float(parts[5])
                                accel_y_kf      = float(parts[6])
                                accel_z_kf      = float(parts[7])
                                roll            = float(parts[8])
                                pitch           = float(parts[9])
                                yaw             = float(parts[10])
                                
                                # Combine sensor data with metadata from folder name
                                row = [
                                    timestamp,
                                    system_activated,
                                    accel_x,
                                    accel_y,
                                    accel_z,
                                    accel_x_kf,
                                    accel_y_kf,
                                    accel_z_kf,
                                    roll,
                                    pitch,
                                    yaw,
                                    trip_date,
                                    distance,
                                    driver_label,
                                    behavior,
                                    road
                                ]
                                combined_data.append(row)
                            except ValueError:
                                print(f"Skipping line (conversion error): {line}")
                                continue

    # Write all the combined data into one CSV file
    with open(output_csv, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(header)  # write header row
        writer.writerows(combined_data)
    print(f"Combined CSV file created at: {output_csv}")

if __name__ == "__main__":
    # Set the path to your dataset's root directory
    root_directory = "C:/Documents/ST AiOT/datasets/UAH-DRIVESET-v1/UAH-DRIVESET-v1"  # <-- Modify this path accordingly
    output_csv_file = "combined_accel_gyro_data.csv"
    
    process_all_folders(root_directory, output_csv_file)


Processing file: C:/Documents/ST AiOT/datasets/UAH-DRIVESET-v1/UAH-DRIVESET-v1\D1\20151110175712-16km-D1-NORMAL1-SECONDARY\RAW_ACCELEROMETERS.txt
Processing file: C:/Documents/ST AiOT/datasets/UAH-DRIVESET-v1/UAH-DRIVESET-v1\D1\20151110180824-16km-D1-NORMAL2-SECONDARY\RAW_ACCELEROMETERS.txt
Processing file: C:/Documents/ST AiOT/datasets/UAH-DRIVESET-v1/UAH-DRIVESET-v1\D1\20151111123124-25km-D1-NORMAL-MOTORWAY\RAW_ACCELEROMETERS.txt
Processing file: C:/Documents/ST AiOT/datasets/UAH-DRIVESET-v1/UAH-DRIVESET-v1\D1\20151111125233-24km-D1-AGGRESSIVE-MOTORWAY\RAW_ACCELEROMETERS.txt
Processing file: C:/Documents/ST AiOT/datasets/UAH-DRIVESET-v1/UAH-DRIVESET-v1\D1\20151111132348-25km-D1-DROWSY-MOTORWAY\RAW_ACCELEROMETERS.txt
Processing file: C:/Documents/ST AiOT/datasets/UAH-DRIVESET-v1/UAH-DRIVESET-v1\D1\20151111134545-16km-D1-AGGRESSIVE-SECONDARY\RAW_ACCELEROMETERS.txt
Processing file: C:/Documents/ST AiOT/datasets/UAH-DRIVESET-v1/UAH-DRIVESET-v1\D1\20151111135612-13km-D1-DROWSY-SECONDARY\R

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the combined CSV file
data = pd.read_csv("combined_accel_gyro_data.csv")

# For classification, we use sensor readings as features
# Here, we select some example features; you can add or remove columns as needed.
features = ["AccelX_Gs", "AccelY_Gs", "AccelZ_Gs", "Roll_deg", "Pitch_deg", "Yaw_deg"]
target = "Behavior"  # Ground truth label from folder name

# Drop rows with missing data in the selected columns
data = data.dropna(subset=features + [target])

# Encode the categorical behavior labels into numeric codes
data[target] = data[target].astype('category')
data['Behavior_code'] = data[target].cat.codes

# Define feature matrix X and target vector y
X = data[features]
y = data['Behavior_code']

# Split the data into training and test sets (70/30 split, stratified by target)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Create and train a Random Forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Evaluate accuracy and print a classification report
acc = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(acc * 100))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 80.83%

Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.74      0.78     23900
           1       0.81      0.80      0.80     29956
           2       0.80      0.86      0.82     39560

    accuracy                           0.81     93416
   macro avg       0.81      0.80      0.80     93416
weighted avg       0.81      0.81      0.81     93416

